In [ ]:
import os
from tensorflow.keras import models, layers, activations, \
    optimizers, utils, losses, initializers, metrics, callbacks

In [ ]:
path = 'C:\Users\disrct\Desktop\Machine_Learning\RedeNeural/train'
epochs = 100
batch_size = 32 #quantidade de dados
patience = 5 #ira analisar o erro durante as épocas se ele não diminuir durante cinco épocas a paciencia acaba e da early stopping
learning_rate = 0.001 #alfa do sgd
model_path = 'C:\Users\disrct\Desktop\Machine_Learning\RedeNeural\train'


In [ ]:
model =  models.Sequential([
    #Camadas de convulsao
    layers.Resizing(56, 56), #muda o tamanho de todas as imagens padronizando elas
    layers.Rescaling(1.0/255),#muda os valores do pixel e deixa entre 0 e 1
    layers.RandomRotation((-0.2, 0.2)), #muda a rotação da imagem, girando ela e vendo de outros angulos

    layers.Conv2D(32, (3, 3), #aplicação dos filtros serão 32 filtros 3x3
        activation =  'relu',
        kernel_initializer =  initializers.RandomNormal()
    ),
    layers.MaxPooling2D((2, 2)), #pega o valor maximo de cada quadrado nessa caso 2x2

    layers.Conv2D(64, (4, 4), #2048 @ 24x24 64 filtros x 32 imgs
        activation =  'relu',
        kernel_initializer =  initializers.RandomNormal()          
    ),
    layers.MaxPooling2D((3, 3)), #2048 @ 8x8

    layers.Flatten(), #pega todos os valores e transforma em um vetor gigante +- 125k

    layers.Dropout(0.2), # a cada época serão dropados 20% dos neuronios

    #Camada de processamento
    layers.Dense(128,
        activation =  'relu',
        kernel_initializer =  initializers.RandomNormal()  
    ),
                  
    layers.Dense(64,
        activation =  'relu',
        kernel_initializer =  initializers.RandomNormal()  
    ),

    layers.Dense(64,
        activation =  'relu',
        kernel_initializer =  initializers.RandomNormal()  
    ),

    layers.Dense(1,
        activation =  'sigmoid',
        kernel_initializer =  initializers.RandomNormal()  
    ),
])

In [ ]:
model.compile(
    optimizer = optimizers.Adam(
        learning_rate = learning_rate
    ),
    loss = losses.BinaryCrossentropy(),
    metrics = [
        metrics.BinaryAccuracy(),
        metrics.Precision(),
        metrics.Recall()
    ]
)

In [ ]:
train = utils.image_dataset_from_directory(
    path,
    validation_split = 0.2,
    subset = "training",
    shuffle = True,
    seed = 42,
    image_size = (224, 224),
    batch_size = batch_size 
)

In [ ]:
test = utils.image_dataset_from_directory(
    path,
    validation_split = 0.2,
    subset = "validation",
    shuffle = True,
    seed = 42,
    image_size = (224, 224),
    batch_size = batch_size 
)

In [ ]:
model.fit(
    train,
    validation_data = test,
    epochs = epochs,
    callbacks = [
        callbacks.EarlyStopping(
            monitor = 'val_loss',
            patience = patience,
            verbose = 1
        )
        # callbacks.ModelCheckpoint(
        #     filepath = model_path,
        #     save_weights_only = False,
        #     monitor = 'loss',
        #     mode = 'min',
        #     save_best_only = True
        # )
    ]
)